In [0]:
from google.colab import files
uploaded = files.upload()

Saving ValidX.npy to ValidX.npy
Saving ValidY.npy to ValidY.npy
Saving X_res.npy to X_res.npy
Saving Y_res.npy to Y_res.npy


In [0]:
files.download("05fscore.h5")



In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout

# setup class weights
class_weight = {0: 1,1:1.}
#class_weight = {0: 0.5,1:10.}

# split data
#trainX,validX,trainY,validY = train_test_split(X,Y,test_size = 0.2)

trainX = numpy.load('X_res.npy',allow_pickle=True)
trainY = numpy.load('Y_res.npy',allow_pickle=True)
validX = numpy.load('ValidX.npy',allow_pickle=True)
validY = numpy.load('ValidY.npy',allow_pickle=True)


# define the keras model
model = Sequential()
model.add(Dense(64, input_dim=7, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(BatchNormalization())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(BatchNormalization())
model.add(Dense(512, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(BatchNormalization())
model.add(Dense(1024, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(BatchNormalization())
model.add(Dense(64, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(BatchNormalization())
model.add(Dense(6, activation='linear'))
model.add(LeakyReLU(alpha=0.1)) 
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='mean_squared_logarithmic_error', optimizer='nadam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(trainX, trainY, epochs=100, batch_size=100, class_weight=class_weight, validation_data=(validX,validY))
# evaluate the model
scores = model.evaluate(validX, validY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# get predictions
pred = model.predict(validX, verbose = True)
binary = numpy.round(pred)
Y_Valid = [float(i) for i in validY]

Train on 85364 samples, validate on 13922 samples
Epoch 1/100
85364/85364 [==============================] - 19s 217us/step - loss: 0.1101 - acc: 0.6245 - val_loss: 0.0562 - val_acc: 0.9563
Epoch 2/100
85364/85364 [==============================] - 14s 169us/step - loss: 0.1065 - acc: 0.6266 - val_loss: 0.0679 - val_acc: 0.9422
Epoch 3/100
85364/85364 [==============================] - 14s 167us/step - loss: 0.0938 - acc: 0.7007 - val_loss: 0.0532 - val_acc: 0.9429
Epoch 4/100
85364/85364 [==============================] - 14s 167us/step - loss: 0.0867 - acc: 0.7326 - val_loss: 0.0448 - val_acc: 0.9550
Epoch 5/100
85364/85364 [==============================] - 14s 166us/step - loss: 0.0849 - acc: 0.7393 - val_loss: 0.0680 - val_acc: 0.8714
Epoch 6/100
85364/85364 [==============================] - 14s 167us/step - loss: 0.0836 - acc: 0.7430 - val_loss: 0.0551 - val_acc: 0.9532
Epoch 7/100
85364/85364 [==============================] - 14s 169us/step - loss: 0.0825 - acc: 0.7463 - val_l

In [0]:
# report
from sklearn.metrics import confusion_matrix,classification_report, f1_score

print(confusion_matrix(Y_Valid,binary))
print(classification_report(Y_Valid, binary))

[[7909 5405]
 [ 337  271]]
              precision    recall  f1-score   support

         0.0       0.96      0.59      0.73     13314
         1.0       0.05      0.45      0.09       608

    accuracy                           0.59     13922
   macro avg       0.50      0.52      0.41     13922
weighted avg       0.92      0.59      0.71     13922



In [0]:
model.save("05fscore.h5")